In [2]:
#중요코드
import requests
import pandas as pd
from datetime import datetime

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver7.csv'

# CSV 파일 읽기
df = pd.read_csv(path)

# movieCd를 저장할 새로운 열 추가
df['movieCd'] = None

# API 키
api_key = "4e50f96e56d711078d42488bdf77cbe0"

# 요청 URL
url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json"

for index, row in df.iterrows():
    movie_title = row['Movie_Title']
    director = row['Director']
    release_year = row['Release_Date'][:4]
    release_date = row['Release_Date'][:4]+row['Release_Date'][5:7]+row['Release_Date'][8:10]
    
    # 요청 파라미터 설정
    params = {
        'key': api_key,
        'curPage': '1',
        'itemPerPage': '10',
        'movieNm': movie_title,
        'openStartDt': release_year
    }
    
    # GET 요청
    response = requests.get(url, params=params)
    
    # 응답 확인
    if response.status_code == 200:
        data = response.json()
        # 영화 목록에서 release_date와 openDt가 같은 영화의 movieCd 추출
        if data['movieListResult']['totCnt'] > 0:
            found_movie_cd = False
            for movie in data['movieListResult']['movieList']:
                if movie['openDt'] == release_date:
                    movieCd = movie['movieCd']
                    df.at[index, 'movieCd'] = movieCd
                    found_movie_cd = True
                    break
            if not found_movie_cd:
                print(f"No matching movie found for title '{movie_title}' with release date '{release_date}'")
        else:
            print(f"No movie found for title '{movie_title}' and director '{director}'")
    else:
        print(f"Error: {response.status_code} for movie title '{movie_title}' and director '{director}'")

# 변경된 데이터프레임을 CSV 파일로 저장
df.to_csv("merged_dataset_add_cd.csv", index=False)

print(f"Updated movie codes have been saved to {path}")


Updated movie codes have been saved to 0530_dataset/merged_dataset.csv


In [7]:
#중요코드
import requests
import json
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver5.csv'

# CSV 파일 읽기
df = pd.read_csv(path)

# movieCd를 저장할 새로운 열 추가
df['first_act'] = None
df['second_act'] = None

# API 키
api_key = "0ce87dc872a5a617a94149263d2bb648"

for index, row in df.iterrows():
    mvcd=row['movieCd']

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json"
    params = {
        'key': api_key,
        'movieCd': mvcd
    }
    
    # GET 요청
    response = requests.get(url, params=params)
    
    # 응답 확인
    if response.status_code == 200:
        data = response.json()
        
        # actors 리스트에서 첫 번째와 두 번째 peopleNm 가져오기
        actors = data['movieInfoResult']['movieInfo']['actors']
        
        if len(actors) >= 2:
            first_actor = actors[0]['peopleNm']
            second_actor = actors[1]['peopleNm']
            df.at[index,'first_act'] = first_actor
            df.at[index,'second_act'] = second_actor
            
            print(f"First Actor: {first_actor}")
            print(f"Second Actor: {second_actor}")
        else:
            print("Less than 2 actors available in the data.")
            first_actor = actors[0]['peopleNm']
            df.at[index,'first_act'] = first_actor
    else:
        print(f"Error: {response.status_code}")

df.to_csv("preprosessing_ver6.csv", index=False)

First Actor: 황정민
Second Actor: 정우성
First Actor: 최민식
Second Actor: 김고은
First Actor: 김무열
Second Actor: 박지환
First Actor: 조 샐다나
Second Actor: 샘 워싱턴
First Actor: 마동석
Second Actor: 이준혁
First Actor: 톰 크루즈
Second Actor: 마일즈 텔러
First Actor: 박해일
Second Actor: 변요한
First Actor: 현빈
Second Actor: 유해진
First Actor: 김혜수
Second Actor: 염정아
First Actor: 김윤석
Second Actor: 백윤식
First Actor: 이정재
Second Actor: 정우성
First Actor: 크리스 프랫
Second Actor: 조 샐다나
First Actor: 톰 크루즈
Second Actor: 헤일리 앳웰
First Actor: 이병헌
Second Actor: 박서준
First Actor: 티모시 샬라메
Second Actor: 칼라 레인
First Actor: 류준열
Second Actor: 유해진
First Actor: 정성화
Second Actor: 김고은
First Actor: 킬리언 머피
Second Actor: 에밀리 블런트
First Actor: 크리스 프랫
Second Actor: 브라이스 달라스 하워드
First Actor: 신시아
Second Actor: 박은빈
First Actor: 크리스 헴스워스
Second Actor: 나탈리 포트만
First Actor: 강하늘
Second Actor: 정소민
First Actor: 레티티아 라이트
Second Actor: 루피타 뇽오
First Actor: 송강호
Second Actor: 이병헌
First Actor: 티모시 샬라메
Second Actor: 젠데이아 콜먼
First Actor: 고경표
Second Actor: 이이경
First Actor: 키아누 리브스
S

In [7]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver5.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['Director_code'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    director = row['Director'].split(',')[0]
    mvnm = row['Movie_Title']  # 영화 제목에서 첫 번째 항목 선택

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json"
    params = {
        'key': api_key,
        'peopleNm': director,
        'itemPerPage' : 100
    }
    
    # GET 요청
    response = requests.get(url, params=params)
    
    # 응답 확인
    if response.status_code == 200:
        data = response.json()
        
        # peopleList에서 감독 정보를 찾기
        for person in data['peopleListResult']['peopleList']:
            if 'filmoNames' in person:
                if mvnm in person['filmoNames']:
                    df.at[index, 'Director_code'] = person['peopleCd']
                    if person['repRoleNm'] != '감독':
                        print(person)
                    break  # 첫 번째 일치하는 결과만 저장
    else:
        print(f"Failed to fetch data for {director}. Status code: {response.status_code}")

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver5_r2.csv', index=False)
print(df.head())


{'peopleCd': '10088291', 'peopleNm': '허명행', 'peopleNmEn': 'HEO Myeong-hoeng', 'repRoleNm': '액션/스턴트', 'filmoNames': "범죄도시4|더 문|황야|용감한 시민|화란|범죄도시3|유령|압꾸정|헌트|범죄도시2|보호자|리멤버|반도|사라진 시간|천문: 하늘에 묻는다|백두산|시동|나를 찾아줘|나쁜 녀석들: 더 무비|걸캅스|돈|어쩌다, 결혼|뺑반|극한직업|성난황소|7년의 밤|마약왕|공작|신과함께-인과 연|독전|챔피언|골든슬럼버|염력|그것만이 내 세상|미옥|대장 김창수|범죄도시|남한산성|불한당: 나쁜 놈들의 세상|보안관|마스터|아수라|덕혜옹주|부산행|난폭한 기록|미씽: 사라진 여자|검사외전|나를 잊지 말아요|대호|극적인 하룻밤|검은 사제들|더 폰|무뢰한|차이나타운|허삼관|기술자들|봄|두근두근 내 인생|신의 한 수|황제를 위하여|우는 남자|돼지 같은 여자|군도: 민란의 시대|보호자|남자가 사랑할 때|롤러코스터|화이: 괴물을 삼킨 아이|감시자들|파파로티|고스톱 살인|마이 라띠마|전설의 주먹|신세계|베를린|용의자X|미쓰GO|차형사|범죄와의 전쟁: 나쁜놈들 전성시대|카운트다운|푸른소금|열여덟, 열아홉|아이들...|헬로우 고스트|황해|초능력자|부당거래|살인의 강|해결사|악마를 보았다|이끼|파괴된 사나이|비밀애|전우치|국가대표|마더|좋은 놈, 나쁜 놈, 이상한 놈 |다찌마와리|중천|뚝방전설|짝패|사랑해, 말순씨|야수와 미녀|분홍신|달콤한 인생|주먹이 운다(Crying Fist)|바람의 파이터|아라한 장풍대작전|태극기 휘날리며|영어완전정복|스캔들-조선남녀상열지사|내츄럴시티|와일드 카드|성냥팔이 소녀의 재림|패밀리|공공의 적|피도 눈물도 없이|이것이 법이다|흑수선|무사|인디안 썸머|그라운드 제로|판도라|남자니까 아시잖아요?|인류 멸망 보고서 '천상의 피조물' |인류 멸망 보고서 '멋진 신세계'|타임리스"}
{'peopleCd': '10057315', 'peopleNm': '이정재', 'peopleNmEn'

In [8]:
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver5_r2.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
print(df['Movie_Title'][df['Director_code'].isna()])
print(df['Director_code'].isna().sum())

Series([], Name: Movie_Title, dtype: object)
0


In [9]:
import requests


# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

 # 요청 URL 및 파라미터
url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json"
params = {
        'key': api_key,
        'peopleCd': 10021341
    }
# GET 요청
response = requests.get(url, params=params)
mvnm='신세계'  
# 응답 확인
if response.status_code == 200:
    data = response.json()

print(data)

{'peopleInfoResult': {'peopleInfo': {'peopleCd': '10021341', 'peopleNm': '마동석', 'peopleNmEn': 'Don LEE', 'sex': '남자', 'repRoleNm': '배우', 'homepages': [], 'filmos': [{'movieCd': '20228797', 'movieNm': '범죄도시4', 'moviePartNm': '배우'}, {'movieCd': '20228797', 'movieNm': '범죄도시4', 'moviePartNm': '제작'}, {'movieCd': '20228797', 'movieNm': '범죄도시4', 'moviePartNm': '기획'}, {'movieCd': '20228797', 'movieNm': '범죄도시4', 'moviePartNm': '각색'}, {'movieCd': '20218800', 'movieNm': '거룩한 밤: 데몬 헌터스', 'moviePartNm': '배우'}, {'movieCd': '20235556', 'movieNm': '황야', 'moviePartNm': '배우'}, {'movieCd': '20226411', 'movieNm': '범죄도시3', 'moviePartNm': '배우'}, {'movieCd': '20226411', 'movieNm': '범죄도시3', 'moviePartNm': '배우'}, {'movieCd': '20226411', 'movieNm': '범죄도시3', 'moviePartNm': '제작'}, {'movieCd': '20226411', 'movieNm': '범죄도시3', 'moviePartNm': '각색'}, {'movieCd': '20205106', 'movieNm': '압꾸정', 'moviePartNm': '배우'}, {'movieCd': '20205106', 'movieNm': '압꾸정', 'moviePartNm': '제작'}, {'movieCd': '20205106', 'movieNm': '압꾸정', 

In [13]:
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver5_r2.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['Director_code'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json"
params = {
        'key': api_key,
        'peopleCd':10058005
    }
    
 # GET 요청
response = requests.get(url, params=params)
    
    # 응답 확인
if response.status_code == 200:
    data = response.json()
        
        
print(data)

{'peopleInfoResult': {'peopleInfo': {'peopleCd': '10058005', 'peopleNm': '이한', 'peopleNmEn': 'LEE Han', 'sex': '남자', 'repRoleNm': '감독', 'homepages': [], 'filmos': [{'movieCd': '20226500', 'movieNm': '달짝지근해: 7510', 'moviePartNm': '감독'}, {'movieCd': '20226500', 'movieNm': '달짝지근해: 7510', 'moviePartNm': '각색'}, {'movieCd': '20185881', 'movieNm': '증인', 'moviePartNm': '감독'}, {'movieCd': '20185881', 'movieNm': '증인', 'moviePartNm': '시나리오(각본)'}, {'movieCd': '20158483', 'movieNm': '오빠생각', 'moviePartNm': '감독'}, {'movieCd': '20158483', 'movieNm': '오빠생각', 'moviePartNm': '각색'}, {'movieCd': '20148844', 'movieNm': '손님', 'moviePartNm': '제작'}, {'movieCd': '20148844', 'movieNm': '손님', 'moviePartNm': '기획'}, {'movieCd': '20149062', 'movieNm': '연애의 맛', 'moviePartNm': '각색'}, {'movieCd': '20142407', 'movieNm': '오늘의 연애', 'moviePartNm': '각색'}, {'movieCd': '20138226', 'movieNm': '우아한 거짓말', 'moviePartNm': '감독'}, {'movieCd': '20138226', 'movieNm': '우아한 거짓말', 'moviePartNm': '각색'}, {'movieCd': '20112695', 'movieNm': 

In [6]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver8.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['di_ca'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    director = row['Director_code']

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json"
    params = {
        'key': api_key,
        'peopleCd': director
    }
    
    # GET 요청
    response = requests.get(url, params=params)
    
    # 응답 확인
    if response.status_code == 200:
        data = response.json()
        
        director_codes = []
        person = data['peopleInfoResult']['peopleInfo']
        if 'filmos' in person:
            for film in person['filmos']:
                if film['moviePartNm'] == '감독':
                    director_codes.append(film['movieNm'])
        
        if director_codes:
            df.at[index, 'di_ca'] = ','.join(director_codes)

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver8_r1.csv', index=False)
print(df.head())


   Rank Movie_Title Release_Date    Box_Office  Box_Office_Share  \
0   1.0       서울의 봄   2023-11-22  1.279270e+11             0.048   
1   2.0          파묘   2024-02-22  1.151211e+11             0.044   
2   3.0       범죄도시4   2024-04-24  1.059487e+11             0.040   
3   4.0   아바타: 물의 길   2022-12-14  1.376477e+11             0.052   
4   5.0       범죄도시3   2023-05-31  1.046877e+11             0.040   

   Cumulative_Box_Office    Audience  Cumulative_Audience  Screens  Showings  \
0           1.279270e+11  13128080.0           13128080.0   2463.0  374533.0   
1           1.151211e+11  11910325.0           11910325.0   2367.0  364429.0   
2           1.059487e+11  11075758.0           11075758.0   2980.0  340400.0   
3           1.376477e+11  10805065.0           10805065.0   2809.0  283037.0   
4           1.046877e+11  10682813.0           10682813.0   2582.0  311915.0   

   ... w3_av_sales first_act_code second_act_code     one_Distributor  \
0  ...   25.442857       10090290    

In [ ]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver5_r3.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['Actor_code_1'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    director = row['Director'].split(',')[0]
    mvnm = row['Movie_Title']  # 영화 제목에서 첫 번째 항목 선택

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json"
    params = {
        'key': api_key,
        'peopleNm': director,
        'itemPerPage' : 100
    }
    
    # GET 요청
    response = requests.get(url, params=params)
    
    # 응답 확인
    if response.status_code == 200:
        data = response.json()
        
        # peopleList에서 감독 정보를 찾기
        for person in data['peopleListResult']['peopleList']:
            if 'filmoNames' in person:
                if mvnm in person['filmoNames']:
                    df.at[index, 'Director_code'] = person['peopleCd']
                    if person['repRoleNm'] != '감독':
                        print(person)
                    break  # 첫 번째 일치하는 결과만 저장
    else:
        print(f"Failed to fetch data for {director}. Status code: {response.status_code}")

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver5_r2.csv', index=False)
print(df.head())


In [4]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver7.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['first_act_code'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    actor = row['first_act']
    mvnm = row['Movie_Title']  # 영화 제목에서 첫 번째 항목 선택

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json"
    params = {
        'key': api_key,
        'peopleNm': actor,
        'itemPerPage' : 100
    }
    
    # GET 요청
    response = requests.get(url, params=params)
    
    # 응답 확인
    if response.status_code == 200:
        data = response.json()
        
        # peopleList에서 감독 정보를 찾기
        for person in data['peopleListResult']['peopleList']:
            if 'filmoNames' in person:
                if mvnm in person['filmoNames']:
                    df.at[index, 'first_act_code'] = person['peopleCd']
                    if person['repRoleNm'] != '배우':
                        print(person)
                    break  # 첫 번째 일치하는 결과만 저장
    else:
        print(f"Failed to fetch data for {actor}. Status code: {response.status_code}")

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver7_r2.csv', index=False)
print(df.head())


{'peopleCd': '10034708', 'peopleNm': '산드라 휠러', 'peopleNmEn': 'Sandra Huller', 'repRoleNm': '', 'filmoNames': '존 오브 인터레스트|추락의 해부|엘리자벳과 나|아임 유어 맨|인 디 아일|토니 에드만|레퀴엠'}
{'peopleCd': '20380214', 'peopleNm': '황윤', 'peopleNmEn': 'Hwang Yun', 'repRoleNm': '감독', 'filmoNames': '수라|잡식가족의 딜레마|어느날 그 길에서|침묵의 숲|작별|광장의 닭'}
{'peopleCd': '10034495', 'peopleNm': '사카모토 류이치', 'peopleNmEn': 'Ryuichi Sakamoto', 'repRoleNm': '음악', 'filmoNames': '류이치 사카모토: 오퍼스|괴물|안녕, 티라노: 영원히, 함께|류이치 사카모토: 에이싱크|류이치 사카모토: 코다|남한산성'}
{'peopleCd': '10048995', 'peopleNm': '엔니오 모리꼬네', 'peopleNmEn': 'Ennio Morricone', 'repRoleNm': '음악', 'filmoNames': '엔니오: 더 마에스트로|타슌가|스탕달 신드롬 |스타 메이커 |러브 어페어|데스티니|언터처블|베니스의 정사|라가비아|레드 소냐 |사하라 |그대 머무는 곳에|누구에게 줄까요'}
{'peopleCd': '20112223', 'peopleNm': '나탈리 포트만', 'peopleNmEn': 'Natalie Portman', 'repRoleNm': '감독', 'filmoNames': '메이 디셈버|토르: 러브 앤 썬더|우먼 인 할리우드|서던 리치: 소멸의 땅|플래니테리엄|헤이데이|재키|제인 갓 어 건|사랑과 어둠의 이야기|나이트 오브 컵스|토르: 다크 월드|토르: 천둥의 신|친구와 연인사이|유어 하이니스|블랙 스완|브라더스|뉴욕 아이 러브 유|천일의 스캔들|마고리엄의 장난감 백화점|고야의 유령|사랑

In [5]:
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver7_r2.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
print(df['Movie_Title'][df['first_act_code'].isna()])
print(df['first_act_code'].isna().sum())

Series([], Name: Movie_Title, dtype: object)
0


In [6]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver7_r2.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['second_act_code'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    actor = row['second_act']
    mvnm = row['Movie_Title']  # 영화 제목에서 첫 번째 항목 선택

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json"
    params = {
        'key': api_key,
        'peopleNm': actor,
        'itemPerPage' : 100
    }
    
    # GET 요청
    response = requests.get(url, params=params)
    
    # 응답 확인
    if response.status_code == 200:
        data = response.json()
        
        # peopleList에서 감독 정보를 찾기
        for person in data['peopleListResult']['peopleList']:
            if 'filmoNames' in person:
                if mvnm in person['filmoNames']:
                    df.at[index, 'second_act_code'] = person['peopleCd']
                    if person['repRoleNm'] != '배우':
                        print(person)
                    break  # 첫 번째 일치하는 결과만 저장
    else:
        print(f"Failed to fetch data for {actor}. Status code: {response.status_code}")

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver7_r3.csv', index=False)
print(df.head())


{'peopleCd': '20258694', 'peopleNm': '박지환', 'peopleNmEn': '', 'repRoleNm': '광고대행', 'filmoNames': '범죄도시4|서울의 봄|화란|타겟|범죄도시3|하루|임금님의 사건수첩|조작된 도시|공조|마스터|형|아수라|고산자, 대동여지도'}
{'peopleCd': '10088663', 'peopleNm': '헤일리 앳웰', 'peopleNmEn': 'Hayley Atwell', 'repRoleNm': '', 'filmoNames': '미션 임파서블: 데드 레코닝 PART ONE|곰돌이 푸 다시 만나 행복해|마블: 슈퍼 히어로 군단의 비밀|올 이즈 바이 마이 사이드|스위니|아이,애나|퍼스트 어벤져|마블의 75년-펄프에서 팝컬처로'}
{'peopleCd': '20112223', 'peopleNm': '나탈리 포트만', 'peopleNmEn': 'Natalie Portman', 'repRoleNm': '감독', 'filmoNames': '메이 디셈버|토르: 러브 앤 썬더|우먼 인 할리우드|서던 리치: 소멸의 땅|플래니테리엄|헤이데이|재키|제인 갓 어 건|사랑과 어둠의 이야기|나이트 오브 컵스|토르: 다크 월드|토르: 천둥의 신|친구와 연인사이|유어 하이니스|블랙 스완|브라더스|뉴욕 아이 러브 유|천일의 스캔들|마고리엄의 장난감 백화점|고야의 유령|사랑해, 파리|브이 포 벤데타|스타워즈 에피소드3 - 시스의복수|프리 존|클로저|가든 스테이트|스타워즈 에피소드 2 : 클론의 습격|노블리|스타워즈:에피소드1 보이지 않는 위험 |여기보다 어딘가에|레옹 디 오리지널|레옹 '}
{'peopleCd': '10078519', 'peopleNm': '클린트 이스트우드', 'peopleNmEn': 'Clint Eastwood', 'repRoleNm': '감독', 'filmoNames': '엔니오: 더 마에스트로|크라이 마초|리차드 쥬얼|라스트 미션|설리: 허드슨강의 기적|아메리칸 스나이퍼|저지 보이즈|내 인생의 마지막 변화구

In [10]:
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver7_r3.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
print(df['Movie_Title'][df['second_act_code'].isna()])
print(df['second_act_code'].isna().sum())

89                 아임 히어로 더 파이널
124                  보 이즈 어프레이드
130                          수라
133                        더 웨일
135         바람 따라 만나리 : 김호중의 계절
136               류이치 사카모토: 오퍼스
137                  사운드 오브 프리덤
138               인생은 뷰티풀: 비타돌체
145                       애프터 양
148                     오토라는 남자
154    슈가│어거스트 디 투어 ‘디-데이’ 더 무비
158                     바튼 아카데미
159                      어른 김장하
164                   마이 샤이니 월드
197                      그대가 조국
Name: Movie_Title, dtype: object
15


In [3]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver7_r3.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['Pc_code'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    maker = row['Production_Company']
    mvnm = row['Movie_Title']  # 영화 제목에서 첫 번째 항목 선택

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/company/searchCompanyList.json"
    params = {
        'key': api_key,
        'companyNm': maker,
        'itemPerPage' : 100
    }
    
    # GET 요청
    response = requests.get(url, params=params)
    
    # 응답 확인
    if response.status_code == 200:
        data = response.json()
        
        # companyList에서 영화사 정보를 찾기
        for company in data['companyListResult']['companyList']:
            if 'filmoNames' in company:
                if mvnm in company['filmoNames']:
                    df.at[index, 'Pc_code'] = company['companyCd']
                    break  # 첫 번째 일치하는 결과만 저장
    else:
        print(f"Failed to fetch data for {maker}. Status code: {response.status_code}")

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver7_r4.csv', index=False)
print(df.head())


   Rank Movie_Title Release_Date    Box_Office  Box_Office_Share  \
0   1.0       서울의 봄   2023-11-22  1.279270e+11             0.048   
1   2.0          파묘   2024-02-22  1.151211e+11             0.044   
2   3.0       범죄도시4   2024-04-24  1.059487e+11             0.040   
3   4.0   아바타: 물의 길   2022-12-14  1.376477e+11             0.052   
4   5.0       범죄도시3   2023-05-31  1.046877e+11             0.040   

   Cumulative_Box_Office    Audience  Cumulative_Audience  Screens  Showings  \
0           1.279270e+11  13128080.0           13128080.0   2463.0  374533.0   
1           1.151211e+11  11910325.0           11910325.0   2367.0  364429.0   
2           1.059487e+11  11075758.0           11075758.0   2980.0  340400.0   
3           1.376477e+11  10805065.0           10805065.0   2809.0  283037.0   
4           1.046877e+11  10682813.0           10682813.0   2582.0  311915.0   

   ... Distributor_Sc first_act second_act w3_rank w1_av_sales w2_av_sales  \
0  ...              A       황정민 

In [10]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver7_r6.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['Distributor_code'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    i=0
    maker = row['one_Distributor']
    mvnm = row['Movie_Title']  # 영화 제목에서 첫 번째 항목 선택
    if maker =="메가박스중앙(주)":
        maker="플러스엠 엔터테인먼트"
        i+=1
        

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/company/searchCompanyList.json"
    params = {
        'key': api_key,
        'companyNm': maker,
        'itemPerPage' : 100
    }
    
    # GET 요청
    response = requests.get(url, params=params)
    
    # 응답 확인
    if response.status_code == 200:
        data = response.json()
        
        # companyList에서 영화사 정보를 찾기
        for company in data['companyListResult']['companyList']:
            if 'filmoNames' in company:
                if mvnm in company['filmoNames'] or i==1:
                    df.at[index, 'Distributor_code'] = company['companyCd']
                    break  # 첫 번째 일치하는 결과만 저장
            print(f"배급사 {maker} 없음")
    else:
        print(f"Failed to fetch data for {maker}. Status code: {response.status_code}")

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver7_r7.csv', index=False)
print(df.head())


   Rank Movie_Title Release_Date    Box_Office  Box_Office_Share  \
0   1.0       서울의 봄   2023-11-22  1.279270e+11             0.048   
1   2.0          파묘   2024-02-22  1.151211e+11             0.044   
2   3.0       범죄도시4   2024-04-24  1.059487e+11             0.040   
3   4.0   아바타: 물의 길   2022-12-14  1.376477e+11             0.052   
4   5.0       범죄도시3   2023-05-31  1.046877e+11             0.040   

   Cumulative_Box_Office    Audience  Cumulative_Audience  Screens  Showings  \
0           1.279270e+11  13128080.0           13128080.0   2463.0  374533.0   
1           1.151211e+11  11910325.0           11910325.0   2367.0  364429.0   
2           1.059487e+11  11075758.0           11075758.0   2980.0  340400.0   
3           1.376477e+11  10805065.0           10805065.0   2809.0  283037.0   
4           1.046877e+11  10682813.0           10682813.0   2582.0  311915.0   

   ... first_act second_act w3_rank w1_av_sales w2_av_sales w3_av_sales  \
0  ...       황정민        정우성       1

In [7]:
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver5_r2.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['Director_code'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json"
params = {
        'key': api_key,
        'peopleCd': 20112223
    }
    
 # GET 요청
response = requests.get(url, params=params)
    
    # 응답 확인
if response.status_code == 200:
    data = response.json()
        
        
print(data)

{'peopleInfoResult': {'peopleInfo': {'peopleCd': '20112223', 'peopleNm': '나탈리 포트만', 'peopleNmEn': 'Natalie Portman', 'sex': '여자', 'repRoleNm': '감독', 'homepages': [], 'filmos': [{'movieCd': '20247053', 'movieNm': '메이 디셈버', 'moviePartNm': '배우'}, {'movieCd': '20224662', 'movieNm': '토르: 러브 앤 썬더', 'moviePartNm': '배우'}, {'movieCd': '20196726', 'movieNm': '우먼 인 할리우드', 'moviePartNm': '배우'}, {'movieCd': '20192341', 'movieNm': '서던 리치: 소멸의 땅', 'moviePartNm': '배우'}, {'movieCd': '20189464', 'movieNm': '플래니테리엄', 'moviePartNm': '배우'}, {'movieCd': '20184122', 'movieNm': '헤이데이', 'moviePartNm': '배우'}, {'movieCd': '20166707', 'movieNm': '재키', 'moviePartNm': '배우'}, {'movieCd': '20187701', 'movieNm': '제인 갓 어 건', 'moviePartNm': '배우'}, {'movieCd': '20166261', 'movieNm': '사랑과 어둠의 이야기', 'moviePartNm': '감독'}, {'movieCd': '20166261', 'movieNm': '사랑과 어둠의 이야기', 'moviePartNm': '배우'}, {'movieCd': '20156982', 'movieNm': '나이트 오브 컵스', 'moviePartNm': '배우'}, {'movieCd': '20138355', 'movieNm': '토르: 다크 월드', 'moviePartNm': 

In [7]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver8_r1.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['Distributors_mv'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    distributor = row['Distributor_code']

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/company/searchCompanyInfo.json"
    params = {
        'key': api_key,
        'companyCd': distributor
    }
    
    # GET 요청
    response = requests.get(url, params=params)
    
    # 응답 확인
    if response.status_code == 200:
        data = response.json()
        
        company_codes = []
        company = data['companyInfoResult']['companyInfo']
        if 'filmos' in company:
            for film in company['filmos']:
                if film['companyPartNm'] == '배급사':
                    company_codes.append(film['movieNm'])
        
        if company_codes:
            df.at[index, 'Distributors_mv'] = ','.join(company_codes)

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver8_r2.csv', index=False)
print(df.head())


   Rank Movie_Title Release_Date    Box_Office  Box_Office_Share  \
0   1.0       서울의 봄   2023-11-22  1.279270e+11             0.048   
1   2.0          파묘   2024-02-22  1.151211e+11             0.044   
2   3.0       범죄도시4   2024-04-24  1.059487e+11             0.040   
3   4.0   아바타: 물의 길   2022-12-14  1.376477e+11             0.052   
4   5.0       범죄도시3   2023-05-31  1.046877e+11             0.040   

   Cumulative_Box_Office    Audience  Cumulative_Audience  Screens  Showings  \
0           1.279270e+11  13128080.0           13128080.0   2463.0  374533.0   
1           1.151211e+11  11910325.0           11910325.0   2367.0  364429.0   
2           1.059487e+11  11075758.0           11075758.0   2980.0  340400.0   
3           1.376477e+11  10805065.0           10805065.0   2809.0  283037.0   
4           1.046877e+11  10682813.0           10682813.0   2582.0  311915.0   

   ... w3_av_sales first_act_code second_act_code     one_Distributor  \
0  ...   25.442857       10090290    

In [5]:
import pandas as pd

# 첫 번째 CSV 파일 경로 (Director_code와 di_ca 열이 있는 파일)
source_file_path = '0530_dataset\preprosessing_ver5_r3.csv'

# 두 번째 CSV 파일 경로 (열을 추가할 대상 파일)
target_file_path = '0530_dataset\preprosessing_ver7_r8.csv'

# 출력 파일 경로 (병합 결과를 저장할 파일)
output_file_path = '0530_dataset\preprosessing_ver7_r9.csv'

# 첫 번째 CSV 파일 읽기 (Director_code와 di_ca 열 추출)
source_df = pd.read_csv(source_file_path, usecols=['Director_code', 'di_ca'])

# 두 번째 CSV 파일 읽기
target_df = pd.read_csv(target_file_path)

# 두 번째 CSV 파일에 Director_code와 di_ca 열 추가 (기본적으로 인덱스를 기준으로 병합)
merged_df = pd.concat([target_df, source_df], axis=1)

# 병합된 데이터프레임을 새로운 CSV 파일로 저장
merged_df.to_csv(output_file_path, index=False)

print(f"'Director_code'와 'di_ca' 열이 추가된 CSV 파일이 저장되었습니다: {output_file_path}")


'Director_code'와 'di_ca' 열이 추가된 CSV 파일이 저장되었습니다: 0530_dataset\preprosessing_ver7_r9.csv


In [8]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver8_r2.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['first_actor_mv'] = None

# API 키
api_key = "f73414c46c8384565dcdb519d1156297"

for index, row in df.iterrows():
    actor = row['first_act_code']

    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json"
    params = {
        'key': api_key,
        'peopleCd': actor
    }
    
    # GET 요청
    response = requests.get(url, params=params)
    
    # 응답 확인
    if response.status_code == 200:
        data = response.json()
        
        actor_codes = []
        person = data['peopleInfoResult']['peopleInfo']
        if 'filmos' in person:
            for film in person['filmos']:
                if film['moviePartNm'] == '배우':
                    actor_codes.append(film['movieNm'])
        
        if actor_codes:
            df.at[index, 'first_actor_mv'] = ','.join(actor_codes)

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver8_r3.csv', index=False)
print(df.head())


   Rank Movie_Title Release_Date    Box_Office  Box_Office_Share  \
0   1.0       서울의 봄   2023-11-22  1.279270e+11             0.048   
1   2.0          파묘   2024-02-22  1.151211e+11             0.044   
2   3.0       범죄도시4   2024-04-24  1.059487e+11             0.040   
3   4.0   아바타: 물의 길   2022-12-14  1.376477e+11             0.052   
4   5.0       범죄도시3   2023-05-31  1.046877e+11             0.040   

   Cumulative_Box_Office    Audience  Cumulative_Audience  Screens  Showings  \
0           1.279270e+11  13128080.0           13128080.0   2463.0  374533.0   
1           1.151211e+11  11910325.0           11910325.0   2367.0  364429.0   
2           1.059487e+11  11075758.0           11075758.0   2980.0  340400.0   
3           1.376477e+11  10805065.0           10805065.0   2809.0  283037.0   
4           1.046877e+11  10682813.0           10682813.0   2582.0  311915.0   

   ... w3_av_sales first_act_code second_act_code     one_Distributor  \
0  ...   25.442857       10090290    

In [12]:
#중요코드
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver8_r3.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['second_actor_mv'] = None

# API 키
api_key = "0ce87dc872a5a617a94149263d2bb648"

for index, row in df.iterrows():
    actor = row['second_act_code']
    if pd.isna(actor) or actor == '':
        continue
    # 요청 URL 및 파라미터
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json"
    params = {
        'key': api_key,
        'peopleCd': int(actor)
    }
    print(actor)
    # GET 요청
    response = requests.get(url, params=params)
    
    # 응답 확인
    if response.status_code == 200:
        data = response.json()
        
        actor_codes = []
        person = data['peopleInfoResult']['peopleInfo']
        if 'filmos' in person:
            for film in person['filmos']:
                if film['moviePartNm'] == '배우':
                    actor_codes.append(film['movieNm'])
        
        if actor_codes:
            df.at[index, 'second_actor_mv'] = ','.join(actor_codes)

# 데이터프레임 저장 또는 확인
df.to_csv('0530_dataset/preprosessing_ver8_r4.csv', index=False)
print(df.head())


10062025.0
20125838.0
10021341.0
10035005.0
20133615.0
20184490.0
20147110.0
10054128.0
10049852.0
10030351.0
10062025.0
10065071.0
10088663.0
20211007.0
20393267.0
10054128.0
20125838.0
10048445.0
10031727.0
20112070.0
20112223.0
20191086.0
10059402.0
10055626.0
10063226.0
20124322.0
10016307.0
10088453.0
10079526.0
20127372.0
10088975.0
20179005.0
20179797.0
20170901.0
10055970.0
20201026.0
10006529.0
10000558.0
20386219.0
10049852.0
20121464.0
10069179.0
20171222.0
20305090.0
20157462.0
10004099.0
10082678.0
10043877.0
20385012.0
10085649.0
20110148.0
10005297.0
20391668.0
20388557.0
20377381.0
20379786.0
20125825.0
10086911.0
10023532.0
10061581.0
10014012.0
20276769.0
10009210.0
10040372.0
20209526.0
10014990.0
20327652.0
10053981.0
20283489.0
20127194.0
10050734.0
10075795.0
10075848.0
20279929.0
10079237.0
10058646.0
20202246.0
20384890.0
20371688.0
10054128.0
10041643.0
10043371.0
20220846.0
10072250.0
10058646.0
20390889.0
10037229.0
10064954.0
20388562.0
10006537.0
20279251.0

In [11]:
import requests
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver5_r2.csv'

# CSV 파일 읽기
df = pd.read_csv(path)
df['Director_code'] = None

# 요청 URL 및 파라미터
url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json"
params = {
        'key': api_key,
        'peopleCd': 10021341
    }
 # GET 요청
response = requests.get(url, params=params)
    
    # 응답 확인
if response.status_code == 200:
    data = response.json()
        
        
print(data)

{'peopleInfoResult': {'peopleInfo': {'peopleCd': '10021341', 'peopleNm': '마동석', 'peopleNmEn': 'Don LEE', 'sex': '남자', 'repRoleNm': '배우', 'homepages': [], 'filmos': [{'movieCd': '20228797', 'movieNm': '범죄도시4', 'moviePartNm': '배우'}, {'movieCd': '20228797', 'movieNm': '범죄도시4', 'moviePartNm': '제작'}, {'movieCd': '20228797', 'movieNm': '범죄도시4', 'moviePartNm': '기획'}, {'movieCd': '20228797', 'movieNm': '범죄도시4', 'moviePartNm': '각색'}, {'movieCd': '20218800', 'movieNm': '거룩한 밤: 데몬 헌터스', 'moviePartNm': '배우'}, {'movieCd': '20235556', 'movieNm': '황야', 'moviePartNm': '배우'}, {'movieCd': '20226411', 'movieNm': '범죄도시3', 'moviePartNm': '배우'}, {'movieCd': '20226411', 'movieNm': '범죄도시3', 'moviePartNm': '배우'}, {'movieCd': '20226411', 'movieNm': '범죄도시3', 'moviePartNm': '제작'}, {'movieCd': '20226411', 'movieNm': '범죄도시3', 'moviePartNm': '각색'}, {'movieCd': '20205106', 'movieNm': '압꾸정', 'moviePartNm': '배우'}, {'movieCd': '20205106', 'movieNm': '압꾸정', 'moviePartNm': '제작'}, {'movieCd': '20205106', 'movieNm': '압꾸정', 

In [8]:
import pandas as pd

# CSV 파일 경로 설정
path = '0530_dataset/preprosessing_ver7_r11.csv'

# CSV 파일 읽기
df = pd.read_csv(path)

# 서로 다른 Movie_Title에 대해 Audience와 Cumulative_Audience가 다른 경우 찾기
different_audience_titles = df[df['Audience'] != df['Cumulative_Audience']]['Movie_Title'].unique()

# 결과 출력
print("Movie Titles with different Audience and Cumulative Audience values:")
for title in different_audience_titles:
    print(title)


Movie Titles with different Audience and Cumulative Audience values:
마녀(魔女) Part2. The Other One
애프터 양
큐어
킹메이커
경관의 피
특송
웨스트 사이드 스토리
어나더 라운드
우연과 상상
